<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">M2.5 – إنشاء تحليل بيانات مناخية قابل لإعادة الإنتاج</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science" dir="ltr">علوم المناخ الحاسوبية</a> <span dir="ltr" style="unicode-bidi: isolate;">|</span> <strong>الدرس السابق</strong> <span dir="ltr" style="unicode-bidi: isolate;">|</span> <strong>الدرس التالي</strong></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>المحتويات</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Creating-a-reproducible-workflow" dir="ltr">إنشاء سير عمل قابل لإعادة الإنتاج</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Our-workflow:-Downloading-the-data-(Step-1)" dir="ltr">سير العمل الخاص بنا: تنزيل البيانات (الخطوة 1)</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Our-workflow:-Data-processing-(Step-2)" dir="ltr">سير العمل الخاص بنا: معالجة البيانات (الخطوة 2)</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#A-reproducible-project&#x27;s-files" dir="ltr">ملفات مشروع قابل لإعادة الإنتاج</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Comparing-multiple-years-of-climate-data" dir="ltr">مقارنة عدة سنوات من البيانات المناخية</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Computing-TOA-radiation" dir="ltr">حساب إشعاع أعلى الغلاف الجوي (TOA)</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Computing-PET" dir="ltr">حساب PET</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Resampling-the-CHIRPS-data" dir="ltr">إعادة أخذ العينات لبيانات CHIRPS</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="#Selecting-part-of-an-xarray-time-series" dir="ltr">اختيار جزء من سلسلة زمنية في xarray</a></li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إنشاء سير عمل قابل لإعادة الإنتاج</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الدرس السابق استخدمنا <span dir="ltr" style="unicode-bidi: isolate;">dask</span> و <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لقراءة مجموعة من ملفات <span dir="ltr" style="unicode-bidi: isolate;">netCDF</span>، ثم قمنا بـ<strong>تطبيق</strong> <strong>دالة متجهة</strong> على كل مصفوفة ضمن سلسلة زمنية. أنتجنا رسمًا يوضح نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span> لمدينة تيارت في الجزائر خلال الأشهر الأولى من عام <span dir="ltr" style="unicode-bidi: isolate;">2024</span> أثناء جفاف شديد.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بحد ذاته، لا يوضح الرسم مدى شدة الجفاف في تيارت. وعلى الرغم من أن الهطول في المنطقة عوّض أقل من 5% من الماء المفقود خلال الأشهر الماضية، فقد يكون ذلك جزءًا من الدورة الموسمية الطبيعية. نحن نعرف أن الفترة من يناير إلى أبريل تعد عادةً فترة أكثر رطوبة نسبيًا في تيارت، لكن يبقى السؤال المطروح: <strong>هل يمكننا مقارنة هذا العام بالسنوات السابقة</strong>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عندما نرغب في تطبيق تحليل مكتمل على مجموعة بيانات جديدة، سواء عبر الزمن أو عبر المكان، فهذه فرصة لتحسين تمثيل سير العمل وجعله أكثر قابلية لإعادة الاستخدام. انظر إلى السكربتين أدناه اللذين يمثلان أول خطوتين في سير عملنا.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سير العمل الخاص بنا: تنزيل البيانات (الخطوة 1)</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد يسمى الملف الأول مثلًا: <strong><span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD_step1_download_MERRA2_data.py</span></strong>. تذكّر أن <span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD</span> يمثل تاريخ اليوم، وهو يساعدنا على <strong>ربط ملفات المخرجات</strong> بالشيفرة التي أنشأتها.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أن ملفات بايثون ذات الامتداد <span dir="ltr" style="unicode-bidi: isolate;">*.py</span> يمكن أن تحتوي على <strong>سلسلة توثيق على مستوى الملف</strong>، وفي المثال أدناه تكون هي سلسلة بايثون متعددة الأسطر التي تبدأ بـ <span dir="ltr" style="unicode-bidi: isolate;">&#x27;&#x27;&#x27;</span>. يجب أن تبدأ سلسلة توثيق الملف في السطر الأول من الملف. وهي مهمة جدًا في سير العمل القابل لإعادة الإنتاج، لأن السطر الأول هو أول مكان ستنظر إليه لفهم هدف الملف.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>من الأعلى إلى الأسفل، لاحظ ما يلي</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وجود <strong>سلسلة توثيق على مستوى الملف</strong> في أعلى السكربت تتضمن معلومات مهمة عن هدف السكربت، ومكان الحصول على المزيد من المعلومات، وكيف يغيّر السكربت نظام الملفات.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وضع جميع عبارات <span dir="ltr" style="unicode-bidi: isolate;">import</span> قرب أعلى السكربت. هذا يوضح للقارئ ما هي الوحدات المطلوبة لتشغيله. لا نريد وضع عبارات الاستيراد في أسفل الملف لأن ذلك يصعّب العثور عليها وقد يؤدي إلى خطأ مفاجئ من نوع ImportError.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحديد المعاملات التي قد تتغير عند تشغيل السكربت بشكل واضح باستخدام أسماء بحروف كبيرة قرب أعلى الملف. على سبيل المثال، <span dir="ltr" style="unicode-bidi: isolate;">DATA_YEAR</span> متغير قد نرغب بتغييره عند تشغيل السكربت عدة مرات. وضعه في أعلى السكربت وبحروف كبيرة يقلل من الحاجة لقراءة كل سطر للعثور عليه.</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سير العمل الخاص بنا: معالجة البيانات (الخطوة 2)</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخطوة التالية هي قراءة ملفات البيانات وحساب إشعاع أعلى الغلاف الجوي (TOA). قد يسمى الملف الثاني مثلًا: <strong><span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD_step2_compute_TOA_radiation.py</span></strong>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذا السكربت يتم حساب الإشعاع وفق صيغة FAO للإشعاع خارج الغلاف الجوي. يمكن قراءة المزيد في الرابط التالي.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://www.fao.org/4/X0490E/x0490e07.htm#radiation" dir="ltr">وثائق FAO لصيغة الإشعاع خارج الغلاف الجوي</a></p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>مرة أخرى، لاحظ ما يلي</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وجود سلسلة توثيق على مستوى الملف وعبارات الاستيراد قرب أعلى السكربت يساعد المستخدمين على تحديد هدف السكربت وما هي وحدات بايثون المطلوبة لتشغيله.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span> تحتوي أيضًا على <strong>سلسلة توثيق على مستوى الدالة</strong> تشرح طريقة عمل الدالة، والمدخلات المطلوبة، وقيمة الإرجاع.</li>
</ul><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>فكّر في السطر الذي يحتوي على</strong> <span dir="ltr" style="unicode-bidi: isolate;">if __name__ == &#x27;__main__&#x27;</span> <strong>وما معناه</strong></p><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن تنفيذ أي ملف بايثون <span dir="ltr" style="unicode-bidi: isolate;">*.py</span> بطريقتين. الأولى تشغيله كسكربت عبر <span dir="ltr" style="unicode-bidi: isolate;">python myscript.py</span>. والثانية استيراده كوحدة عبر <span dir="ltr" style="unicode-bidi: isolate;">import myscript</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند تشغيل ملف بايثون، ينشئ بايثون المتغير <span dir="ltr" style="unicode-bidi: isolate;">__name__</span> الذي يحدد اسم الوحدة. إذا شُغّل الملف كسكربت فإن قيمة <span dir="ltr" style="unicode-bidi: isolate;">__name__</span> تصبح <span dir="ltr" style="unicode-bidi: isolate;">&#x27;__main__&#x27;</span>. لذلك يمكن استخدام هذا الشرط لاختبار ما إذا كان الكود يُنفّذ كسكربت أو كوحدة مستوردة.</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ملفات مشروع قابل لإعادة الإنتاج</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">هناك خطوات أخرى في تحليلنا، لكن نتركها لك كتدريب لكتابة سكربتات قابلة لإعادة الاستخدام ومُحكمة التوثيق مثل الأمثلة أعلاه. قد يبدو مجلد المشروع النهائي على النحو التالي.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><img src="./assets/M2_file_tree_workflow.png" alt="M2 file tree workflow" dir="ltr"></p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تعليق الشكل يوضح مثالًا لبنية ملفات مشروع سير عمل قابل لإعادة الإنتاج.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 <strong>لاحظ أن كل ملف داخل مجلد scripts يتضمن كلمات مثل step1 و step2 و step3</strong> لتحديد ترتيب تنفيذ سير العمل. هذا يساعد القارئ على معرفة ترتيب التنفيذ، وقد يساعدك أنت أيضًا إذا عدت إلى المشروع بعد أشهر ولم تتذكر ما الذي قمت به.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مقارنة عدة سنوات من البيانات المناخية</h2><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في النهاية نريد الإجابة عن سؤال الدرس السابق. كيف تقارن نسبة الهطول إلى PET لشتاء 2024 مع نسبة شتاء 2023 في تيارت.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن لسير العمل القابل لإعادة الإنتاج أعلاه أن يجيب عن هذا السؤال. كما سنرى أيضًا كيف نجيب عنه ضمن جلسة بايثون تفاعلية داخل دفتر Jupyter هذا. إذا كنا قد نزّلنا بيانات MERRA-2 لكل من 2023 و2024، فنحن جاهزون لتحميل البيانات باستخدام <span dir="ltr" style="unicode-bidi: isolate;">xarray.open_mfdataset()</span>.</p>
</div>

In [ ]:
import xarray as xr
from matplotlib import pyplot

ds_chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')
ds_merra2 = xr.open_mfdataset('data_raw/MERRA2/*.nc4', chunks = 'auto')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يحتوي سكربت بايثون <span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD_step2_compute_TOA_radiation.py</span> على دالة مفيدة وقابلة لإعادة الاستخدام هي <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span>. <strong>كيف يمكننا استخدام هذه الدالة دون نسخ ولصق تعريفها</strong> في سكربت آخر أو داخل هذا الدفتر.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بشكل عام، ينبغي تجنب وجود أكثر من نسخة واحدة لأي دالة. أحد الأسباب الرئيسية لكتابة الدوال هو تجنب تكرار الشيفرة.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">👈 كما أشرنا أعلاه، فإن استخدام الشرط <span dir="ltr" style="unicode-bidi: isolate;">if __name__ == &#x27;__main__&#x27;:</span> يساعد على التمييز بين تشغيل الملف كسكربت أو استيراده كوحدة. طالما يمكننا رؤية السكربت <span dir="ltr" style="unicode-bidi: isolate;">YYYYMMDD_step2_compute_TOA_radiation.py</span> في شجرة الملفات، فسنتمكن من استيراده كوحدة. أدناه نستخدم مكتبة os لعرض ملفات الدليل الحالي. يمكننا رؤية وجود مجلد scripts ضمن هذا الدليل.</p>
</div>

In [ ]:
import os

# Display the files in the current directory
files = os.listdir('.') # The single dot "." indicates the current directory
files.sort()
files

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وداخل مجلد scripts يوجد سكربت بايثون الذي نريد استيراده كوحدة.</p>
</div>

In [ ]:
os.listdir('scripts')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كيف نفعل ذلك؟ لأن كلًا من المجلدات وملفات بايثون ذات الامتداد *.py يمكن التعامل معها كوحدات، فكل ما نحتاجه هو كتابة عبارة استيراد لوحدة كما في المثال التالي.</p>
</div>

In [ ]:
from scripts.YYYYMMDD_step2_compute_TOA_radiation import toa_radiation_wrapper

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أننا استوردنا اسمًا واحدًا فقط من هذا السكربت، وهو الدالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation_wrapper()</span>، مما يمنحنا وصولًا إلى الدالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span>. وبسبب كتابتنا سلسلة توثيق مفيدة على مستوى الدالة، يمكننا معرفة طريقة الاستخدام عبر استدعاء <span dir="ltr" style="unicode-bidi: isolate;">help()</span> على الدالة.</p>
</div>

In [ ]:
help(toa_radiation_wrapper)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب إشعاع أعلى الغلاف الجوي (TOA)</h3>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما في السابق، لحساب إشعاع أعلى الغلاف الجوي نريد إنشاء شبكة خطوط عرض لكي نتمكن من توجيه خطوة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span> على نحو متجه.</p>
</div>

In [ ]:
lat_grid = ds_merra2.lat.values.reshape(1, 361, 1)\
    .repeat(ds_merra2.lon.size, axis = 2)\
    .repeat(ds_merra2.time.size, axis = 0)
lat_grid.shape

In [ ]:
ds_merra2['lat_grid'] = (('time', 'lat', 'lon'), lat_grid)
ds_merra2

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ونحتاج إلى اليوم من السنة، وهو متاح أصلًا كـ <span dir="ltr" style="unicode-bidi: isolate;">ds_merra2[&#x27;time.dayofyear&#x27;]</span>.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وبذلك نكون جاهزين لحساب إشعاع أعلى الغلاف الجوي.</p>
</div>

In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
toa_rad = toa_radiation_wrapper(ds_merra2) * 0.408
toa_rad.sel(time = '2024-05-01').plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب PET</h3>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وسنقوم كذلك باقتطاع بيانات MERRA-2 لمنطقة تيارت.</p>
</div>

In [ ]:
toa_rad.shape

In [ ]:
import numpy as np

ds_merra2['toa_radiation'] = toa_rad

# Select just the Tiaret region
merra2_tiaret = ds_merra2.sel(lon = -1.32, lat = 35.37, method = 'nearest')

# Compute PET using the Hargreaves equation
merra2_tiaret_pet = 0.0023 * merra2_tiaret['toa_radiation'] * np.sqrt(
    merra2_tiaret['T2MMAX'] - merra2_tiaret['T2MMIN']) * (merra2_tiaret['T2MMEAN'] + 17.8)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إعادة أخذ العينات لبيانات CHIRPS</h3>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أخيرًا نحتاج إلى معالجة بيانات الهطول الشهرية من CHIRPS وتحويلها إلى هطول يومي لمنطقة تيارت، كما فعلنا في الدرس السابق.</p>
</div>

In [ ]:
chirps_tiaret = ds_chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret_daily = chirps_tiaret.resample(time = 'D').nearest().mean(['x', 'y']) / 30
chirps_tiaret_daily

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اختيار جزء من سلسلة زمنية في xarray</h3><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن نحن جاهزون لمقارنة عامي 2023 و2024.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إذا افترضنا أن الدليل <span dir="ltr" style="unicode-bidi: isolate;">data_raw/MERRA2</span> يحتوي البيانات التي نزّلناها لكل من 2023 و2024، فإن مجموعة البيانات <span dir="ltr" style="unicode-bidi: isolate;">ds_merra2</span> ستتضمن بيانات العامين. لقد رأينا كيف نختار تاريخًا محددًا باستخدام النمط <span dir="ltr" style="unicode-bidi: isolate;">sel(time = &#x27;YYYY-MM-DD&#x27;)</span>. لكن كيف نختار كل البيانات الخاصة بسنة معينة أو شهر معين أو نطاق من التواريخ.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كل بُعد في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> من نوع <span dir="ltr" style="unicode-bidi: isolate;">datetime64[ns]</span> يتيح مُلحِقًا للوصول إلى خصائص الوقت، ويتم الوصول إليه عبر الخاصية <span dir="ltr" style="unicode-bidi: isolate;">dt</span>.</p>
</div>

In [ ]:
merra2_tiaret_pet.time.dt

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html" dir="ltr">يمكن استخدام خاصية dt لاقتطاع سلسلة زمنية</a></p>
</div>

In [ ]:
merra2_tiaret_pet.time.dt.year

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">على سبيل المثال، لاختيار البيانات الخاصة بعام <span dir="ltr" style="unicode-bidi: isolate;">2023</span> فقط.</p>
</div>

In [ ]:
# Select only those data points along the time access where time.year is in a list of years
merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([2023]))

In [ ]:
ratios = []
precip = []
for year in [2023, 2024]:
    pet_this_year = merra2_tiaret_pet.sel(time = merra2_tiaret_pet.time.dt.year.isin([year]))
    precip_this_year = chirps_tiaret_daily.sel(time = chirps_tiaret_daily.time.dt.year.isin([year]))
    # Select the first 122 days
    pet_this_year = pet_this_year.isel(time = slice(0, 122))
    precip_this_year = precip_this_year.isel(time = slice(0, 122))
    # NOTE: We must use the .values attribute because these are from two different datasets
    ratios.append(100 * precip_this_year.values / pet_this_year.values)
    precip.append(precip_this_year.values)

In [ ]:
pyplot.figure(figsize = (12, 5))
for y, year in enumerate([2023, 2024]):
    pyplot.plot(pet_this_year.time, ratios[y], label = year)
    
pyplot.ylabel('Precipitation-to-PET Ratio (mm day-1)')
pyplot.legend()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يشير الرسم أعلاه إلى أن الظروف الهيدرولوجية في بداية 2024 ليست مختلفة كثيرًا عن العام السابق. لماذا أدت هذه الظروف الجافة الموسمية إلى <strong>جفاف اجتماعي اقتصادي</strong> في 2024 ولم تؤدِّ إلى ذلك في 2023.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما رأينا عند رسم نسب الهطول إلى PET على المدى الطويل باستخدام بيانات TerraClimate، كان هطول موسم الرطوبة أقل من المعتاد لعدة سنوات. أحد التفسيرات للجفاف الاجتماعي الاقتصادي في 2024 هو أن الأثر التراكمي لعدة سنوات من انخفاض الهطول خلق ظروفًا أشد في 2024 ولم تكن واضحة بنفس الدرجة في 2023.</p><p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا إعادة تصور الانخفاض طويل الأمد في هطول الشتاء باستخدام بيانات CHIRPS. وعلى الرغم من ارتفاع نسبي في الهطول خلال 2022، فمن الواضح أن هطول الشتاء كان منخفضًا بشكل غير معتاد منذ 2019.</p>
</div>

In [ ]:
chirps_tiaret.sel(time = chirps_tiaret.time.dt.month.isin([1,2,3,4,5])).groupby('time.year').sum().mean(['x', 'y']).plot()
pyplot.ylabel('January - May Precipitation (mm)')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">موارد إضافية</h2><ul>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://docs.xarray.dev/en/stable/generated/xarray.core.accessor_dt.DatetimeAccessor.html" dir="ltr">توثيق xarray.core.accessor_dt.DatetimeAccessor</a></li>
</ul>
</div>